In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ViT(nn.Module):
    def __init__(self, image_size, patch_size, num_classes, dim, depth, heads, mlp_dim, dropout_rate, channels=3):
        super().__init__()
        self.patch_size = patch_size

        # 이미지는 패치로 분할되고, 각 패치는 Transformer에 입력될 수 있도록 임베딩되어야 합니다.
        num_patches = (image_size // patch_size) ** 2
        patch_dim = channels * patch_size ** 2
        self.patch_to_embedding = nn.Linear(patch_dim, dim)

        # 클래스 토큰을 추가합니다. 이 토큰은 분류를 위해 사용됩니다.
        self.cls_token = nn.Parameter(torch.randn(1, 1, dim))

        # 위치 임베딩은 Transformer 모델에 시퀀스의 순서 정보를 제공합니다.
        self.pos_embedding = nn.Parameter(torch.randn(1, num_patches + 1, dim))

        # Transformer 인코더를 정의합니다.
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=dim, nhead=heads, dim_feedforward=mlp_dim, dropout=dropout_rate, activation='gelu'),
            num_layers=depth
        )

        # 분류를 위한 MLP 헤드입니다.
        self.mlp_head = nn.Sequential(
            nn.Linear(dim, mlp_dim),
            nn.GELU(),
            nn.Dropout(dropout_rate),
            nn.Linear(mlp_dim, num_classes)
        )

    def forward(self, x):
        b, c, h, w = x.shape

        # 이미지를 패치로 분할하고 임베딩합니다.
        x = x.reshape(b, c, h // self.patch_size, self.patch_size, w // self.patch_size, self.patch_size)
        x = x.transpose(2, 4).flatten(2)
        x = self.patch_to_embedding(x)

        # 클래스 토큰과 위치 임베딩을 추가합니다.
        cls_tokens = self.cls_token.expand(b, -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)
        x += self.pos_embedding

        # Transformer를 통과시킵니다.
        x = self.transformer(x)

        # 분류를 위해 첫 번째 토큰 (클래스 토큰)만 사용합니다.
        x = x[:, 0]

        return self.mlp_head(x)

# 예시 사용
vit = ViT(image_size=256, patch_size=32, num_classes=10, dim=1024, depth=6, heads=8, mlp_dim=2048, dropout_rate=0.1)


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [47]:
import torchvision.models as models

class HybridViT(nn.Module):
    def __init__(self, num_classes, dim, depth, heads, mlp_dim, dropout_rate, backbone_model=models.resnet50(pretrained=True)):
        super().__init__()

        # CNN 기반 백본 모델을 사용합니다.
        self.backbone = backbone_model
        self.backbone.fc = nn.Identity()  # 마지막 FC 레이어는 사용하지 않습니다.

        # 위치 임베딩은 백본 모델의 출력 특징 맵 크기에 맞춰 조정해야 합니다.
        num_patches = 196  # 예시: ResNet의 경우 14x14 특징 맵이 출력될 수 있습니다.
        self.pos_embedding = nn.Parameter(torch.randn(1, num_patches + 1, dim))

        # 클래스 토큰 및 Transformer 인코더는 동일합니다.
        self.cls_token = nn.Parameter(torch.randn(1, 1, dim))
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=dim, nhead=heads, dim_feedforward=mlp_dim, dropout=dropout_rate, activation='gelu'),
            num_layers=depth
        )

        # 분류를 위한 MLP 헤드는 동일합니다.
        self.mlp_head = nn.Sequential(
            nn.Linear(dim, mlp_dim),
            nn.GELU(),
            nn.Dropout(dropout_rate),
            nn.Linear(mlp_dim, num_classes)
        )

    def forward(self, x):
        # 백본 모델을 통해 특징 맵을 추출합니다.
        x = self.backbone(x)
        x = x.view(x.size(0), -1, 2, 1)

        # 추출된 특징 맵을 Transformer의 입력 형식에 맞게 변환합니다.
        b, _, h, w = x.shape
        x = x.flatten(2).transpose(1, 2)

        # 클래스 토큰과 위치 임베딩을 추가합니다.
        cls_tokens = self.cls_token.expand(b, -1, -1)
        print(cls_tokens.shape, x.shape)
        x = torch.cat((cls_tokens, x), dim=1)
        x += self.pos_embedding

        # Transformer를 통과시킵니다.
        x = self.transformer(x)

        # 분류를 위해 첫 번째 토큰 (클래스 토큰)만 사용합니다.
        x = x[:, 0]

        return self.mlp_head(x)

# 예시 사용
hybrid_vit = HybridViT(num_classes=10, dim=1024, depth=6, heads=8, mlp_dim=2048, dropout_rate=0.1)
test_images = torch.randn(2, 3, 224, 224)

# 모델을 통해 테스트 이미지 전달
with torch.no_grad():
    output = hybrid_vit(test_images)

print(output.shape)  # 최종 출력 형상 출력


torch.Size([2, 1, 1024]) torch.Size([2, 2, 1024])


RuntimeError: The size of tensor a (3) must match the size of tensor b (197) at non-singleton dimension 1

In [20]:
backbone = models.resnet50(pretrained=True)
# backbone.avgpool = nn.Identity()
# backbone.fc = nn.Identity()
backbone

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [12]:
# 임의의 테스트 데이터 생성
batch_size = 4
img_height, img_width = 224, 224
test_images = torch.randn(batch_size, 3, img_height, img_width)

# 모델을 통해 테스트 데이터 전달
with torch.no_grad():
    # 백본 모델을 통한 특징 추출
    features = hybrid_vit.backbone(test_images)
    print("후 CNN 특징 맵의 형상:", features.shape)

    # 특징 맵을 Transformer의 입력 형식으로 변환
    b, _, h, w = features.shape
    features = features.flatten(2).transpose(1, 2)
    print("Transformer 입력 형태 변환 후의 형상:", features.shape)

    # 클래스 토큰과 위치 임베딩 추가
    cls_tokens = hybrid_vit.cls_token.expand(b, -1, -1)
    x = torch.cat((cls_tokens, features), dim=1)
    x += hybrid_vit.pos_embedding
    print("클래스 토큰 및 위치 임베딩 추가 후의 형상:", x.shape)

    # Transformer 통과
    x = hybrid_vit.transformer(x)
    print("Transformer 통과 후의 형상:", x.shape)

    # 분류를 위해 첫 번째 토큰만 사용
    x = x[:, 0]
    print("클래스 토큰 추출 후의 형상:", x.shape)

    # MLP 헤드를 통과하여 최종 결과 도출
    logits = hybrid_vit.mlp_head(x)
    print("최종 분류 결과의 형상:", logits.shape)


후 CNN 특징 맵의 형상: torch.Size([4, 100352])


ValueError: not enough values to unpack (expected 4, got 2)

In [13]:
# 임의의 테스트 데이터 생성 (배치 크기 1, 채널 3, 이미지 크기 224x224)
test_images = torch.randn(1, 3, 224, 224)

# 모델을 통해 테스트 데이터 전달
with torch.no_grad():
    output = hybrid_vit(test_images)
    print("최종 출력 형상:", output.shape)

ValueError: not enough values to unpack (expected 4, got 2)

In [1]:
import torch
import torch.nn as nn
import torchvision.models as models

class HybridViT(nn.Module):
    def __init__(self, num_classes=1000, image_size=224, patch_size=1, emb_size=768):
        super(HybridViT, self).__init__()

        # 클래스 변수로 emb_size 저장
        self.emb_size = emb_size

        # 사전 훈련된 ResNet50 모델을 사용하여 특징 맵을 추출합니다.
        self.cnn = models.resnet50(pretrained=True)
        self.cnn = nn.Sequential(*list(self.cnn.children())[:-2])  # 마지막 두 계층 제거

        # CNN 마지막 블록의 출력 채널 수를 얻습니다.
        if isinstance(self.cnn[-1], nn.modules.container.Sequential):
            cnn_out_channels = self.cnn[-1][-1].conv3.out_channels
        else:
            cnn_out_channels = self.cnn[-1].conv1.out_channels

        # 패치 임베딩을 위한 컨볼루션 레이어 설정 (패치 크기 1x1)
        self.patch_emb = nn.Conv2d(cnn_out_channels, emb_size, kernel_size=patch_size)

        # 분류 토큰 및 위치 임베딩 초기화
        self.cls_token = nn.Parameter(torch.zeros(1, 1, emb_size))
        self.pos_embedding = nn.Parameter(torch.randn(1, num_patches + 1, dim))

        # Transformer 인코더 레이어
        self.transformer = nn.TransformerEncoderLayer(d_model=emb_size, nhead=8)

        # 최종 분류를 위한 선형 레이어
        self.head = nn.Linear(emb_size, num_classes)

    def forward(self, x):
        # CNN을 통해 특징 맵 추출
        x = self.cnn(x)

        # 패치 임베딩 프로젝션 적용
        x = self.patch_emb(x)
        x = x.flatten(2).transpose(1, 2)  # 특징 맵을 펼치고, Transformer 형식으로 변환

        # 위치 임베딩의 크기를 설정
        if self.pos_embed is None:
            self.num_patches = x.shape[1]
            self.pos_embed = nn.Parameter(torch.zeros(1, self.num_patches + 1, self.emb_size))

        # 분류 토큰 추가 및 위치 임베딩 적용
        cls_token = self.cls_token.expand(x.shape[0], -1, -1)
        x = torch.cat((cls_token, x), dim=1)
        x += self.pos_embed

        # Transformer 인코더 적용
        x = self.transformer(x)

        # 최종적으로 선형 레이어를 통해 분류
        x = self.head(x[:, 0])
        return x

# 모델 인스턴스 생성
model = HybridViT()

# 더미 이미지 데이터 생성 (예: 배치 크기 1, 채널 3, 크기 224x224)
image = torch.randn(1, 3, 224, 224)  

# 모델에 이미지를 입력하고 결과 출력
output = model(image)
output  # 클래스별 확률을 나타내는 출력값을 확인합니다.


/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


tensor([[-5.7796e-02, -3.0435e-01,  1.8525e-01,  1.1514e-01,  1.0378e-03,
          2.2246e-01, -6.2290e-01,  9.2540e-01,  1.9607e-01,  6.4851e-02,
         -8.0616e-01,  4.6046e-01,  2.3427e-01,  6.3772e-01,  3.1046e-01,
         -6.4819e-01,  8.5056e-02, -4.9830e-01,  1.0569e-01, -1.4349e-01,
          1.8110e-01,  8.3484e-01, -4.8748e-01, -7.3032e-01,  8.3258e-01,
         -2.1776e-01, -9.8833e-01, -1.7072e-01, -2.5503e-01, -1.3897e-01,
          1.3533e-01, -4.0996e-01, -5.4686e-02, -8.7365e-01, -1.1466e+00,
         -4.0644e-01,  6.9038e-01,  3.0300e-01,  7.9072e-02, -4.8798e-01,
          9.4769e-01,  3.5820e-01, -6.9957e-02,  4.4079e-01, -1.6465e-02,
         -1.8087e-01,  3.2531e-01, -1.0300e+00,  3.4123e-01,  4.0831e-01,
         -4.8030e-01,  4.5952e-01,  1.0688e+00,  3.2649e-01,  1.6197e-01,
          4.1529e-01,  6.5238e-01, -3.3665e-02, -2.3757e-01, -6.7546e-02,
         -7.5127e-01, -6.4708e-01,  2.3200e-01,  1.2811e-01,  4.3578e-01,
         -2.8260e-01, -4.0929e-01, -2.